In [ ]:
import os
import glob
import pandas as pd
import requests
import zipfile

# -----------------------------
# Paths
# -----------------------------
# Define file paths for station overview, extracted files, processed data, metadata, and downloads
station_overview_path = r"K:\Fluvial\Meteorological_Data\Rainfall\OPW\scripts\station_overview_table_250217.csv"
extract_base_path     = r"K:\Fluvial\Meteorological_Data\Rainfall\OPW\extracted"
processed_path        = r"K:\Fluvial\Meteorological_Data\Rainfall\OPW\processed"
metadata_path         = r"K:\Fluvial\Meteorological_Data\Rainfall\OPW\metadata"
download_path         = r"K:\Fluvial\Meteorological_Data\Rainfall\OPW\zips"

# Make folders if they don't exist
os.makedirs(extract_base_path, exist_ok=True)
os.makedirs(processed_path, exist_ok=True)
os.makedirs(metadata_path, exist_ok=True)
os.makedirs(download_path, exist_ok=True)

# -----------------------------
# Load station overview
# -----------------------------
# Read CSV containing station numbers to process
station_overview = pd.read_csv(station_overview_path)

# -----------------------------
# Loop through stations
# -----------------------------
# For each station:
#   - Create extraction folder
#   - Download ZIP if missing
#   - Extract ZIP
#   - Process CSV for metadata and rainfall values
for station in station_overview["Station"]:
    station_str = str(station)
    print(f"\nProcessing station: {station_str}")

    # Create folder for extracted files
    extract_path = os.path.join(extract_base_path, station_str)
    os.makedirs(extract_path, exist_ok=True)

    # -----------------------------
    # Download ZIP if not already present
    # -----------------------------
    zip_file = os.path.join(download_path, f"{station_str}.zip")
    if not os.path.exists(zip_file):
        url = f"https://waterlevel.ie/hydro-data/data/internet/stations/0/{station_str}/Precip/Rainfall_complete.zip"
        print(f"Downloading {station_str}...")
        try:
            r = requests.get(url)
            r.raise_for_status()
            with open(zip_file, 'wb') as f:
                f.write(r.content)
            print(f"Downloaded zip for station {station_str}")
        except Exception as e:
            print(f"Failed to download {station_str}: {e}")
            continue

    # -----------------------------
    # Extract ZIP
    # -----------------------------
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Extracted to {extract_path}")
    except Exception as e:
        print(f"Failed to extract {station_str}: {e}")
        continue

    # -----------------------------
    # Locate tsvalues CSV
    # -----------------------------
    # Find the CSV file inside extracted folder
    csv_files = glob.glob(os.path.join(extract_path, "*.csv"))
    if not csv_files:
        print(f"No CSV found for station {station_str}")
        continue
    csv_file_path = csv_files[0]
    print(f"Processing file: {csv_file_path}")

    # -----------------------------
    # Read metadata
    # -----------------------------
    # Extract station ID, name, latitude, longitude from CSV header
    tsvalues_meta = pd.read_csv(csv_file_path, nrows=5, sep=';', index_col=0, names=['Attribute', 'Value'])
    metadata_stat = {
        'ID': tsvalues_meta.loc['#station_no','Value'],
        'Name': tsvalues_meta.loc['#station_name','Value'],
        'Latitude': tsvalues_meta.loc['#station_latitude','Value'],
        'Longitude': tsvalues_meta.loc['#station_longitude','Value']
    }
    print("Metadata:", metadata_stat)

    # -----------------------------
    # Read rainfall time series
    # -----------------------------
    # Separate rainfall values and quality codes
    # Convert timestamps to datetime and set as index
    df = pd.read_csv(csv_file_path, sep=';', skiprows=8)
    df_values  = df[['#Timestamp','Value']].copy()
    df_quality = df[['#Timestamp','Quality Code']].copy()
    df_values['#Timestamp'] = pd.to_datetime(df_values['#Timestamp'])
    df_quality['#Timestamp'] = pd.to_datetime(df_quality['#Timestamp'])
    df_values.set_index('#Timestamp', inplace=True)
    df_quality.set_index('#Timestamp', inplace=True)

    # Filter out missing values marked as -99
    df_values = df_values[df_values['Value'] != -99]

    # -----------------------------
    # Hourly aggregation
    # -----------------------------
    # Sum 15-min values into hourly totals
    # Mark hour as -99 if any 15-min value is invalid
    hourly_sum = df_values['Value'].resample('h').sum()
    hourly_count_valid = df_quality['Quality Code'].resample('h').apply(lambda x: (x==254).all())
    hourly_sum[~hourly_count_valid] = -99

    # Ensure complete hourly index
    full_hour_index = pd.date_range(start=hourly_sum.index.min(),
                                    end=hourly_sum.index.max(),
                                    freq='h')
    hourly_sum = hourly_sum.reindex(full_hour_index, fill_value=-99)

    df_hourly = hourly_sum.to_frame(name='rainfall').reset_index()
    df_hourly.rename(columns={'index':'time'}, inplace=True)
    df_hourly['stno'] = metadata_stat['ID']
    df_hourly = df_hourly[['time','stno','rainfall']]

    # Save hourly CSV
    path_hourly = os.path.join(processed_path, f"{station_str}_hourly.csv")
    df_hourly.to_csv(path_hourly, index=False)
    print(f"Saved hourly data: {path_hourly} | Rows: {df_hourly.shape[0]}")

    # -----------------------------
    # Daily aggregation 09Z→09Z
    # -----------------------------
    # Sum rainfall from 09:00 of one day to 08:59:59 of next day
    daily_records_09Z = []
    first_day = df_values.index.min().normalize() + pd.Timedelta(hours=9)
    last_day  = df_values.index.max().normalize() + pd.Timedelta(hours=9)

    current_day = first_day
    while current_day <= last_day:
        day_start = current_day
        day_end   = day_start + pd.Timedelta(hours=24)

        df_day_values  = df_values.loc[day_start:day_end - pd.Timedelta(seconds=1)]
        df_day_quality = df_quality.loc[day_start:day_end - pd.Timedelta(seconds=1)]

        if not df_day_values.empty and (df_day_quality['Quality Code'] == 254).all():
            value = df_day_values['Value'].sum()
        else:
            value = -99

        daily_records_09Z.append({'date': day_start.date(), 'stno': metadata_stat['ID'], 'rainfall': value})
        current_day += pd.Timedelta(days=1)

    df_daily_09Z = pd.DataFrame(daily_records_09Z)
    path_daily_09Z = os.path.join(processed_path, f"{station_str}_daily_09Z.csv")
    df_daily_09Z.to_csv(path_daily_09Z, index=False)
    print(f"Saved daily 09Z→09Z data: {path_daily_09Z} | Rows: {df_daily_09Z.shape[0]}")

    # -----------------------------
    # Daily aggregation 00Z→23:59
    # -----------------------------
    # Sum rainfall from 00:00 to 23:59:59 UTC
    daily_records_UTC = []
    first_day_UTC = df_values.index.min().normalize()
    last_day_UTC  = df_values.index.max().normalize()

    current_day = first_day_UTC
    while current_day <= last_day_UTC:
        day_start = current_day
        day_end   = day_start + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

        df_day_values  = df_values.loc[day_start:day_end]
        df_day_quality = df_quality.loc[day_start:day_end]

        if not df_day_values.empty and (df_day_quality['Quality Code'] == 254).all():
            value = df_day_values['Value'].sum()
        else:
            value = -99

        daily_records_UTC.append({'date': day_start.date(), 'stno': metadata_stat['ID'], 'rainfall': value})
        current_day += pd.Timedelta(days=1)

    df_daily_UTC = pd.DataFrame(daily_records_UTC)
    path_daily_UTC = os.path.join(processed_path, f"{station_str}_daily_UTC.csv")
    df_daily_UTC.to_csv(path_daily_UTC, index=False)
    print(f"Saved daily 00Z→23:59 data: {path_daily_UTC} | Rows: {df_daily_UTC.shape[0]}")

    # -----------------------------
    # Save metadata
    # -----------------------------
    # Save station metadata as CSV
    metadata_output_path = os.path.join(metadata_path, f"{station_str}_metadata.csv")
    pd.DataFrame([metadata_stat]).to_csv(metadata_output_path, index=False)
    print(f"Saved metadata: {metadata_output_path}")



Processing station: 80709
Extracted to K:\Fluvial\Meteorological_Data\Rainfall\OPW\extracted\80709
Processing file: K:\Fluvial\Meteorological_Data\Rainfall\OPW\extracted\80709\tsvalues.csv
Metadata: {'ID': '80709', 'Name': 'Ballinatona Water Treatment Plant', 'Latitude': '52.25785307', 'Longitude': '-9.05015031'}
Saved hourly data: K:\Fluvial\Meteorological_Data\Rainfall\OPW\processed\80709_hourly.csv | Rows: 161128
Saved daily 09Z→09Z data: K:\Fluvial\Meteorological_Data\Rainfall\OPW\processed\80709_daily_09Z.csv | Rows: 6715
Saved daily 00Z→23:59 data: K:\Fluvial\Meteorological_Data\Rainfall\OPW\processed\80709_daily_UTC.csv | Rows: 6715
Saved metadata: K:\Fluvial\Meteorological_Data\Rainfall\OPW\metadata\80709_metadata.csv

Processing station: 81903
Extracted to K:\Fluvial\Meteorological_Data\Rainfall\OPW\extracted\81903
Processing file: K:\Fluvial\Meteorological_Data\Rainfall\OPW\extracted\81903\tsvalues.csv
Metadata: {'ID': '81903', 'Name': 'Ballingeary', 'Latitude': '51.856816',